In [1]:
import pandas as pd

rides = pd.read_parquet('../data/transformed/validated_rides_2024_01.parquet')

rides.head(10)

,pickup_datetime,pickup_location_id
0,2024-01-01 00:57:55,186
1,2024-01-01 00:03:00,140
2,2024-01-01 00:17:06,236
3,2024-01-01 00:36:38,79
4,2024-01-01 00:46:51,211
5,2024-01-01 00:54:08,148
6,2024-01-01 00:49:44,138
7,2024-01-01 00:30:40,246
8,2024-01-01 00:26:01,161
9,2024-01-01 00:28:08,113


In [3]:
# Add a column that contains the rounded hour 
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

C:\Users\ijato\AppData\Local\Temp\ipykernel_22876\1403050559.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')


,pickup_datetime,pickup_location_id,pickup_hour
0,2024-01-01 00:57:55,186,2024-01-01 00:00:00
1,2024-01-01 00:03:00,140,2024-01-01 00:00:00
2,2024-01-01 00:17:06,236,2024-01-01 00:00:00
3,2024-01-01 00:36:38,79,2024-01-01 00:00:00
4,2024-01-01 00:46:51,211,2024-01-01 00:00:00
...,...,...,...
2964619,2024-01-31 23:45:59,107,2024-01-31 23:00:00
2964620,2024-01-31 23:13:07,114,2024-01-31 23:00:00
2964621,2024-01-31 23:19:00,211,2024-01-31 23:00:00
2964622,2024-01-31 23:07:23,107,2024-01-31 23:00:00


In [5]:
#Count number of rides per hour and pickup location
agg_rides = rides.groupby(['pickup_hour','pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0:'rides'}, inplace = True)
agg_rides

,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,4,25
1,2024-01-01 00:00:00,7,4
2,2024-01-01 00:00:00,9,1
3,2024-01-01 00:00:00,10,6
4,2024-01-01 00:00:00,12,4
...,...,...,...
77525,2024-01-31 23:00:00,260,2
77526,2024-01-31 23:00:00,261,12
77527,2024-01-31 23:00:00,262,9
77528,2024-01-31 23:00:00,263,53


In [7]:
#Make a complete time series by adding record for location & hour pair that had no rides

from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i])
    
    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output
     


In [8]:
agg_rides_all_slots = add_missing_slots(agg_rides)

C:\Users\ijato\AppData\Local\Temp\ipykernel_22876\2040504109.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
100%|██████████| 260/260 [00:00<00:00, 450.00it/s]


In [9]:
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()
     

In [10]:
#Plotting time series for Central Park - Location 43
plot_rides(agg_rides_all_slots, locations=[43])

In [11]:
#Save time series data to disk
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2024_01.parquet')